<a href="https://colab.research.google.com/github/ShihchiaYJMd/stable-diffusion-v1/blob/main/demo_sdv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ShihchiaYJMd/stable-diffusion-v1.git

In [ ]:
!pip install pytorch-lightning
# prompt: 我要下载https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.ckpt

# !wget https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt

import requests
from pathlib import Path

# 文件的URL
url = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt"

# 指定目标目录
CKPT_PATH = Path("ckpt")
CKPT_PATH.mkdir(parents=True, exist_ok=True)  # 创建目录（如果不存在）

# 发送GET请求
response = requests.get(url, stream=True)

# 检查请求是否成功
if response.status_code == 200:
    # 定义文件的保存路径
    save_path = CKPT_PATH / "v1-5-pruned-emaonly.ckpt"

    # 打开一个文件用于写入二进制数据
    with open(save_path, "wb") as file:
        # 逐块写入文件
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"文件下载完成，保存路径: {save_path}")
else:
    print(f"下载失败，状态码: {response.status_code}")

In [ ]:
# 列出 ckpt 目录中的文件
!ls ckpt

In [ ]:
import sys
import os

# Add the paths to your model_loader and pipeline files to the system path.
# Replace 'path/to/your/stable-diffusion-v1' with the actual path
# to the cloned repository. The example below assumes the files
# are in the 'stable-diffusion-v1' directory.
sys.path.append('/content/stable-diffusion-v1/sd1/sdv1')

try:
    import model_loader
    import pipeline
    print("Successfully imported model_loader and pipeline.")

except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Please double-check that the paths are correct "
          "and the files exist in the specified directories.")
    print("Current sys.path:", sys.path)

In [ ]:
import model_loader
import pipeline
from PIL import Image
from transformers import CLIPTokenizer
import torch

DEVICE = 'cpu'

ALLOW_CUDA = False
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = 'cuda'
elif (torch.backends.mps.is_built() or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = 'mps'
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer("/content/stable-diffusion-v1/sd1/data/vocab.json", merges_file="/content/stable-diffusion-v1/sd1/data/merges.txt")
model_file = "ckpt/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

# Text-to-Image
prompt = "a pretty beautiful woman with a sword"
uncond_prompt = ""
do_cfg = True
cfg_scale = 7

# Image-to-Image
input_image = None
image_path = "/content/stable-diffusion-v1/sd1/images/qiuzhen_yau.jpg"
# input_image = Image.open(image_path).convert("RGB")
strength = 0.9

sampler = 'ddpm'
num_inference_steps = 50
seed = 42

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    models=models,
    seed=seed,
    device=DEVICE,
    idle_device='cpu',
    tokenizer=tokenizer
)

Image.fromarray(output_image)

save_path = "output1.png"
Image.fromarray(output_image).save(save_path)